In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/판례_본문.xlsx')

In [ ]:
print(df.head())

In [ ]:
df_2 = pd.read_excel('/content/drive/MyDrive/df_semi_final.xlsx')

In [ ]:
df_2_num = df_2[['사건번호']]

print(df_2_num.head())

In [ ]:
# df_1에서 사건번호가 df_2_num의 사건번호와 일치하는 데이터를 추출
df_3 = df_1[df_1['사건번호'].isin(df_2_num['사건번호'])]

# 결과 확인
print(df_3.head())

In [ ]:
df_2_selected = df_2[['사건번호', '판결유형']]

In [ ]:
# df_3에 df_2_selected와 사건번호 기준으로 병합하여 판결유형 업데이트
df_3_updated = df_3.merge(df_2_selected, on='사건번호', how='left', suffixes=('', '_new'))

# 판결내용이 있는 경우 df_2의 판결내용으로 대체
df_3_updated['판결유형'] = df_3_updated['판결유형_new'].fillna(df_3_updated['판결유형'])

# 필요 없는 판결내용_new 열 삭제
df_3_updated.drop(columns=['판결유형_new'], inplace=True)

# 결과 확인
print(df_3_updated.head())

In [ ]:
# 필요한 열만 남기기
df_4 = df_updated[['사건명', '사건번호', '사건종류명', '판결유형', '판시사항', '판결요지', '판례내용']]

# 결과 확인
print(df_4.head())
print(df_4.columns)

In [ ]:
# 특수 문자 및 불필요한 공백 제거 함수
def clean_text(text):
    # 텍스트가 비어있는 경우 빈 문자열 반환
    if pd.isna(text):
        return ''
    # 특수 문자 제거 (필요시 조정 가능)
    text = re.sub(r'[^\w\s]', '', text)
    # 다중 공백을 단일 공백으로 대체
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 판시사항, 판결요지, 판례내용 열에 대해 전처리 적용
df_4['판시사항'] = df_4['판시사항'].apply(clean_text)
df_4['판결요지'] = df_4['판결요지'].apply(clean_text)
df_4['판례내용'] = df_4['판례내용'].apply(clean_text)

# 중복된 데이터 제거 (선택사항)
df_4.drop_duplicates(subset=['사건명', '사건종류명', '판결유형', '판시사항', '판결요지', '판례내용'], inplace=True)

# 전처리 후 결과 확인
print(df_4.head())

In [ ]:
df_4.to_excel('df_final_real.xlsx', index=False)
files.download('df_final_real.xlsx')

# **토큰화 작업**

In [ ]:
# GPU가 사용 가능한지 확인하고 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BERT 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=13)
model.to(device)

In [ ]:
# 레이블 매핑을 사건 종류에 따라 다르게 적용하는 함수 정의
label_map = {
    '민사_승소': 0, '민사_패소': 1, '민사_기각': 2,
    '형사_기각': 3, '징역': 4, '벌금': 5, '무혐의': 6,
    '가사_승소': 7, '가사_패소': 8, '가사_기각': 9,
    '세무_승소': 10, '세무_패소': 11, '세무_기각': 12
}

In [ ]:
df_4 = df_4[df_4['판결유형'].isin(label_map.keys())]

In [ ]:
# 텍스트 데이터 전처리 및 토큰화 함수 정의 (tqdm 버전)
def preprocess_and_tokenize(data, column_name):
    tokenized_data = []
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Tokenizing"):
        text = str(row[column_name])
        # 전처리: 불필요한 공백 제거
        text = " ".join(text.split())
        # 토큰화
        tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)
        tokenized_data.append(tokens)
    return tokenized_data

In [ ]:
# 판례내용 토큰화
df_4['판례내용_tokens'] = preprocess_and_tokenize(df_4, '판례내용')

In [ ]:
df_5 = df_4

# **임베딩 작업**

In [ ]:
# BERT 모델과 토크나이저 로드
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)
model.eval()

In [ ]:
# 임베딩 생성 함수 정의
def generate_embedding_from_tokens(tokens_list):
    tokens_tensor = torch.tensor([tokens_list]).to(device)
    with torch.no_grad():
        output = model(input_ids=tokens_tensor, attention_mask=(tokens_tensor > 0).long())
    embedding = output.last_hidden_state[:, 0, :].cpu().numpy().flatten()
    return embedding

In [ ]:
# 임베딩 생성 및 데이터프레임에 추가
embeddings = []
for tokens in df['판례내용_tokens']:
    if isinstance(tokens, str):
        tokens = eval(tokens)
    embeddings.append(generate_embedding_from_tokens(tokens))

# df에 임베딩 열 추가
df_5['임베딩'] = embeddings

In [ ]:
# 쉼표를 추가하는 함수 정의
def add_commas_to_numbers_in_column(value):
    if isinstance(value, str):
        # 숫자와 숫자 사이의 공백을 쉼표로 변환
        return re.sub(r'(?<=[\d\.\-eE])\s+(?=[\d\.\-eE])', ', ', value)
    return value

# '임베딩' 열에 쉼표 추가
if '임베딩' in df.columns:
    df_5['임베딩'] = df_5['임베딩'].apply(add_commas_to_numbers_in_column)

output_file_path = 'df_real_token_with_embeddings_with_commas.xlsx'
df_5.to_excel(output_file_path, index=False)

In [ ]:
files.download('df_real_token_with_embeddings_with_commas.xlsx')